<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="1000px">

# Cubeviz with NIRSpec data - JWebbinar 2024

### Author: Jesse Averbukh | Software Engineer at STScI
### Date: May 13, 2024 

## Introduction
This tutorial will cover the following features:
- Line lists
- Line analysis
- Redshift estimation
- Moment maps
- Spectrum when hovering over spaxel
- Spectral extraction
- Unit conversion

The data used in this webinar is provided courtesy of MAST and the analysis is based on the paper "GOALS-JWST: Gas Dynamics and Excitation in NGC 7469 Revealed by NIRSpec" ( https://iopscience.iop.org/article/10.3847/1538-4357/ad2a50/pdf ) by Marina Bianchin et al. (2024)

We will start with our imports:

In [ ]:
# This is where we import the cubeviz configuration
from jdaviz import Cubeviz

# For accessing the file from MAST and loading it into a temporary directory
import tempfile
# from astroquery.mast import Observations
# import warnings

# For creating spectral subsets
from glue.core.roi import XRangeROI

# For creating spatial subsets
from glue.core.roi import CircularROI, RectangularROI

# For loading line lists
from jdaviz.core.linelists import load_preset_linelist

# For creating a custom line list
from astropy.table import QTable
import astropy.units as u

In [ ]:
# Create an instance of cubeviz
cubeviz = Cubeviz()

In [ ]:
cubeviz.show('sidecar:right')

In [ ]:
fn = 'jw01328-o018_t010_nirspec_g235h-f170lp_s3d.fits'
# data_dir = tempfile.gettempdir()

# result = Observations.download_file(f"mast:JWST/product/{fn}", local_path=f'{data_dir}/{fn}')

# We will use this path for the webinar
staged_path = "/home/shared/preloaded-fits/jdaviz_2024/staged_fits_webbinar_2024/"

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    cubeviz.load_data(f'{staged_path}/{fn}')

In [ ]:
# Retrieve viewer object
spec_viewer = cubeviz.viewers['spectrum-viewer']._obj

# Highlight the H2 1-0S(1) line with a spectral subset
spec_viewer.apply_roi(XRangeROI(2.1544, 2.1588))

In [ ]:
# This creates a new spectral subset
spec_viewer.toolbar.active_tool = spec_viewer.toolbar.tools['bqplot:xrange']

# Highlight the Paschen-alpha line with a spectral subset
spec_viewer.apply_roi(XRangeROI(1.9033, 1.9085))

In [ ]:
# Load H2 line lists and plot the H 1-0S(1) line
ll = cubeviz.plugins['Line Lists']._obj
ll.open_in_tray()
ll.available_lists
cubeviz.load_line_list(load_preset_linelist('H2'))
cubeviz.erase_spectral_lines()
cubeviz.plot_spectral_line('H 1-0S(1)')

In [ ]:
# Load H-Paschen-Brackett line lists and plot the HI 4-3 line
ll = cubeviz.plugins['Line Lists']._obj
ll.open_in_tray()
ll.available_lists
cubeviz.load_line_list(load_preset_linelist('H-Paschen-Brackett'))
cubeviz.erase_spectral_lines()
cubeviz.plot_spectral_line('HI 4-3')
cubeviz.plot_spectral_line('H 1-0S(1)')

In [ ]:
# Look at statistics for the H2 line
la = cubeviz.plugins['Line Analysis']
la.open_in_tray()
la.spectral_subset = 'Subset 1'
la.continuum = 'Surrounding'

In [ ]:
# Select line in order to find redshift of galaxy by comparing rest wavelength
# to the observed wavelength
print(la._obj.line_items)
la._obj.selected_line = 'H 1-0S(1) 2.121'

In [ ]:
# Estimated redshift
la._obj.selected_line_redshift

In [ ]:
# Set line lists to estimated redshift
ll.rs_redshift = la._obj.selected_line_redshift

Now that we have the spectral emissions highlighted, we can create moment maps for each of them to get a better sense of what is happening in the galaxy. We will start with a moment 0 map, which will show us the concentration of the particular element in the galaxy.

In [ ]:
# Create a moment map of 0 for the H2 emission line
mm = cubeviz.plugins['Moment Maps']
mm.open_in_tray()
mm.spectral_subset = 'Subset 1'
mm.continuum = 'Surrounding'
mm.n_moment = 0
mm.add_results.label = 'moment-0-h2'
mm.add_results.viewer = 'uncert-viewer'
mm.calculate_moment()

In [ ]:
# Create a moment map of 0 for the Paschen-alpha emission line
mm = cubeviz.plugins['Moment Maps']
mm.spectral_subset = 'Subset 2'
mm.continuum = 'Surrounding'
mm.n_moment = 0
mm.add_results.label = 'moment-0-pa'
mm.add_results.viewer = 'flux-viewer'
mm.calculate_moment()

The paper makes some observations about the increased amounts of Paschen-alpha corresponding to star-forming regions in the north, west, and southern areas of the outer ring. There also appears to be a slight trail of this element going from the nucleus to the ring just south east of the nucleus.

We can also see that there is a high concentration of H2 in the nucleus. The paper asserts that this shows evidence of an AGN, because the high concentration of H2 is consistent with a nuclear radio jet that the authors hypothesize is leading to increased star formation in the galaxy. We will now create moment maps with a moment of 1 to see the flux-weighted centroid of the Paschen-alpha line. This will show us the velocity field in the galaxy at that wavelength.

In [ ]:
# Create a moment map of 1 for the Paschen-alpha emission line
mm = cubeviz.plugins['Moment Maps']
mm.spectral_subset = 'Subset 2'
mm.continuum = 'Surrounding'
mm.n_moment = 1
mm.reference_wavelength = 1.9065
mm.add_results.label = 'moment-1-pa'
mm.add_results.viewer = 'flux-viewer'
mm.calculate_moment()

In [ ]:
# Create a moment map of 2 for the Paschen-alpha emission line
mm = cubeviz.plugins['Moment Maps']
mm.spectral_subset = 'Subset 2'
mm.continuum = 'Surrounding'
mm.n_moment = 2
mm.reference_wavelength = 1.9065
mm.add_results.label = 'moment-2-pa'
mm.add_results.viewer = 'uncert-viewer'
mm.calculate_moment()

In [ ]:
po = cubeviz.plugins['Plot Options']
po.open_in_tray()
po.viewer = 'flux-viewer'
po.layer = 'moment-1-pa'
po.image_colormap.value = 'rainbow'

In [ ]:
po.viewer = 'uncert-viewer'
po.layer = 'moment-2-pa'
po.stretch_preset.value = 90
po.image_colormap.value = 'rainbow'

Positive values in the flux and uncertainty viewers correspond to redshifted areas and negative values correspond to blueshifted areas. This means the galaxy is rotating in a counter-clockwise direction. We can find the minimum and maximum values to get an estimate of the speed of rotation.

For the moment 2 map, we can see the outer ring is relatively calm with values close to the minimum threshold of the detector, while the nucleus is very turbulent. This is another sign of a nuclear radio jet coming from the AGN in the nucleus.

In [ ]:
# Create spatial subset around large Paschen-alpha area
flux_viewer = cubeviz.viewers['flux-viewer']._obj
flux_viewer.toolbar.active_tool = flux_viewer.toolbar.tools['bqplot:rectangle']
flux_viewer.apply_roi(RectangularROI(25, 30, 41, 50))

In [ ]:
# Create spatial subset around the nucleus
flux_viewer.toolbar.active_tool = flux_viewer.toolbar.tools['bqplot:truecircle']
flux_viewer.apply_roi(CircularROI(xc=23.5, yc=29.2, radius=5))

We can see that Subset 3 is highlighting one of the strongest regions of star formation. Subset 4 is highlighting the strong line flux map in the nucleus. We can use the Spectral Extraction plugin to see how the nucleus looks with a conical aperture.

In [ ]:
# We use a conical aperture for the nucleus because it is a PSF (point spread function)
se = cubeviz.plugins['Spectral Extraction']
se.open_in_tray()
se.aperture = 'Subset 4'
se.wavelength_dependent = True
se.aperture_method = 'Exact'
se.function = 'Sum'
se.collapse_to_spectrum()